<a href="https://colab.research.google.com/github/burakemretetik/makemore/blob/main/makemore_mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open("names.txt", "r").read().splitlines()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [6]:
# build the vocabulary of characters
chars = sorted(list(set("".join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi["."] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [14]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one
X, Y = [], []

for w in words[:5]:

    print('\n', w)
    context = [0] * block_size
    for ch in w + ".":
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)


 emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .

 olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .

 ava
... ---> a
..a ---> v
.av ---> a
ava ---> .

 isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .

 sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [38]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [42]:
C = torch.randn((27,2))
C[5]

tensor([ 2.0395, -0.8465])

In [45]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [46]:
W1 = torch.randn((6,100))
b1 = torch.randn(100)

In [48]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [50]:
h

tensor([[ 0.4159, -1.0000,  0.9988,  ...,  0.7225,  0.5152, -0.8256],
        [ 0.9226, -1.0000,  0.9995,  ..., -0.7251,  0.1894,  0.1682],
        [-0.9899, -0.9999,  0.9671,  ...,  1.0000,  0.9461, -1.0000],
        ...,
        [-0.9860,  0.7724,  0.9903,  ...,  0.7523,  0.9649, -1.0000],
        [-0.9587, -0.2467,  0.9622,  ..., -0.8192,  0.9270,  0.9932],
        [-0.9518, -0.7895, -1.0000,  ...,  0.9980, -0.9115,  0.7281]])

In [53]:
h.shape

torch.Size([32, 100])

In [54]:
W2 = torch.randn((100,27))
b2 = torch.randn(27)

In [55]:
logits = h @ W2 + b2

In [56]:
logits.shape

torch.Size([32, 27])

In [57]:
counts = logits.exp()

In [58]:
prob = counts / counts.sum(1, keepdims=True)

In [59]:
prob.shape

torch.Size([32, 27])